In [2]:
%pwd

'c:\\Users\\Suel.Abbasi\\Desktop\\DSA\\projects\\end to end ml project\\end-to-end-ML-Project-with-mlFlow\\research'

In [3]:
import os
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Suel.Abbasi\\Desktop\\DSA\\projects\\end to end ml project\\end-to-end-ML-Project-with-mlFlow'

In [5]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/saGit1990/end-to-end-ML-Project-with-mlFlow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'fafd6a840e364a6ae6d5357b678a3f3f86b2bc39'

In [ ]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path 
    metric_file_name: Path 
    all_params: dict 
    target_column: str 
    mlflow_uri: str

In [27]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [28]:
class ConfigurationManager:
    def __init__(
        self,
        config_path = CONFIG_FILE_PATH,
        params_path = PARAMS_FILE_PATH,
        schema_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.schema = read_yaml(schema_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path= config.model_path,
            metric_file_name=config.metric_file_name,
            all_params= self.params,
            target_column= self.schema.TARGET_COLUMN.name,
            mlflow_uri= 'https://dagshub.com/saGit1990/end-to-end-ML-Project-with-mlFlow.mlflow'
        )
        
        return model_evaluation_config

In [29]:
import os
import pandas as pd 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse 
import mlflow 
import mlflow.sklearn 
import numpy as np 
import joblib
from mlProject import logger
import dagshub

In [30]:
class ModelEvaluation:
    def __init__(self, config = ModelEvaluationConfig):
        self.config = config 
        
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_absolute_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual, pred) 
        return rmse, mae , r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        test_x = test_data.drop([self.config.target_column], axis = 1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        dagshub.init(repo_owner='saGit1990', repo_name= 'end-to-end-ML-Project-with-mlFlow', mlflow=True)
        
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            
            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)
            logger.info(type(rmse))
            # saving the metrics as local file
            scores = {'rmse': rmse, 'mae': mae, 'r2': r2}
            save_json(path = Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metrics(scores)
            
            if tracking_url_type_store != 'file':
                # register the model
                mlflow.sklearn.log_model(model, 'model', registered_model_name='ElastNetModel')
            else:
                mlflow.sklearn.log_model(model,'model')
            

In [31]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-03-01 04:12:31,114: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-01 04:12:31,118: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-01 04:12:31,123: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-01 04:12:31,125: INFO: common: created directory at: artifacts]
[2025-03-01 04:12:31,130: INFO: common: created directory at: artifacts/model_evaluation]
[2025-03-01 04:12:31,985: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/saGit1990/end-to-end-ML-Project-with-mlFlow "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "saGit1990/end-to-end-ML-Project-with-mlFlow"

[2025-03-01 04:12:31,986: INFO: helpers: Initialized MLflow to track repo "saGit1990/end-to-end-ML-Project-with-mlFlow"]


Repository saGit1990/end-to-end-ML-Project-with-mlFlow initialized!

[2025-03-01 04:12:31,986: INFO: helpers: Repository saGit1990/end-to-end-ML-Project-with-mlFlow initialized!]
[2025-03-01 04:12:32,405: INFO: 2234356749: <class 'numpy.float64'>]
[2025-03-01 04:12:32,415: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/03/01 04:12:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElastNetModel' already exists. Creating a new version of this model...
2025/03/01 04:12:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElastNetModel, version 2
Created version '2' of model 'ElastNetModel'.


🏃 View run clumsy-gull-254 at: https://dagshub.com/saGit1990/end-to-end-ML-Project-with-mlFlow.mlflow/#/experiments/0/runs/9893df7a67d44ef28486eb048ea4928e
🧪 View experiment at: https://dagshub.com/saGit1990/end-to-end-ML-Project-with-mlFlow.mlflow/#/experiments/0
